In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

## 1. Análisis de Boxes por Cada tipo de Consulta

En este análisis se quiere calcular la cantidad de Boxes para Consultas son necesarios para el
INT. Para esto, se utilizará la base de datos de HIS (aunque se contrastará con TrackCare) para
identificar la cantidad de consultas por diagnóstico.

La lógica del análisis es el siguiente:

1. Obtener la cantidad de consultas en un periodo (ejemplo: Año, 22000 Consultas al año).
2. Obtener la cantidad de días laborales en el periodo (ejemplo: En el año 2022, 251 días laborales)
3. Calcular la cantidad de consultas que se deben atender por día. La fórmula para calcularlo es
la siguiente:

$ ConsultasPorDía = \frac{CantidadDeConsultasEnUnPeriodo}{DíasLaboralesEnUnPeriodo}$

4. Calcular la cantidad de Boxes necesario para suplir las Consultas por día. En este caso se asume
que **cada día laboral tiene 9 horas hábiles** para atender consultas, y que se **atienden 3 consultas
por hora**. Lo anterior significaría que **1 Box de Consultas puede realizar 27 Consultas al día** 
(Capacidad de Box). Por lo tanto:

$ BoxesNecesarios = \frac{ConsultasPorDía}{CapacidadDeBox}$

Donde:

$ CapacidadDeBox = (HorasHabilesEnDia * ConsultasPorHora)$

In [3]:
df_his = pd.read_csv(
    "../data/processed/datos_limpios_diagnosticos.csv", encoding="latin-1", sep=";"
)
df_his["fecha_atencion"] = pd.to_datetime(df_his["fecha_atencion"])

In [8]:
cantidad_de_consultas = df_his.groupby(df_his["fecha_atencion"].dt.year)[
    "codigo_reserva_atencion"
].count()

In [17]:
DIAS_LABORALES_POR_ANIO = 260
HORAS_POR_DIA_LABORAL = 9
CONSULTAS_POR_HORA = 3
CAPACIDAD_BOX = HORAS_POR_DIA_LABORAL * CONSULTAS_POR_HORA

In [18]:
cantidad_de_consultas / (DIAS_LABORALES_POR_ANIO * CAPACIDAD_BOX)

fecha_atencion
2016    5.436610
2017    5.855128
2018    6.049858
2019    5.616097
2020    4.077350
2021    5.517379
2022    5.259117
2023    0.000570
Name: codigo_reserva_atencion, dtype: float64